Regression Model: Drafter or Not

In [1]:
# import libraries
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import re

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LinearRegression
import tensorflow as tf

In [2]:
raw1 = pd.read_csv('draft_v02_df - IF DRAFTED.csv')

In [3]:
raw1.head()

,player_id,year,player_name,position_combine,School,height,weight,forty_yard,vertical_leap,bench_press,broad_jump,three_cone,shuttle_run,nfl_team,position_nfl,position_nfl_standard,IF_DRAFTED,draft_round,draft_pick
0,2000JohnAbra,2000,John Abraham,OLB,South Carolina,6 ft 4 in,252.0,4.55,NaN,NaN,NaN,NaN,NaN,NYJ,DE,DE,1,1.0,13.0
1,2000ShauAlex,2000,Shaun Alexander,RB,Alabama,6 ft 0 in,218.0,4.58,NaN,NaN,NaN,NaN,NaN,SEA,RB,RB,1,1.0,19.0
2,2000DarnAlfo,2000,Darnell Alford,OT,Boston Col.,6 ft 4 in,334.0,5.56,25.0,23.0,94.0,8.48,4.98,KAN,G,G,1,6.0,188.0
3,2000KyleAlla,2000,Kyle Allamon,TE,Texas Tech,6 ft 2 in,253.0,4.97,29.0,NaN,104.0,7.29,4.49,NaN,NaN,NaN,0,NaN,NaN
4,2000RashAnde,2000,Rashard Anderson,CB,Jackson State,6 ft 2 in,206.0,4.55,34.0,NaN,123.0,7.18,4.15,CAR,DB,DB,1,1.0,23.0


In [4]:
#Data Exploration
num_nans1 = raw1.isna().sum()
print('Count of missing values for each columns:')
print(num_nans1)

Count of missing values for each columns:
player_id                   0
year                        0
player_name                 0
position_combine            0
School                      0
height                     29
weight                     24
forty_yard                474
vertical_leap            1750
bench_press              2588
broad_jump               1823
three_cone               2891
shuttle_run              2788
nfl_team                 4278
position_nfl             4278
position_nfl_standard    4278
IF_DRAFTED                  0
draft_round              4278
draft_pick               4278
dtype: int64


In [5]:
raw1.dropna(subset = 'height', axis=0, inplace=True)

In [6]:
# change height string into inches (int)
str_height = raw1['height']
def inches(string):
    split_list = re.split(r'\s+', string)
    feet = pd.to_numeric(split_list[0])
    inches = pd.to_numeric(split_list[2])
    total = feet*12 + inches
    return total
    
inches_height = [inches(h) for h in str_height]
inches_height

raw1['height'] = inches_height

In [7]:
# Split our preprocessed data into our features and target arrays
X = raw1[['year', 'height', 'weight', 'forty_yard', 'vertical_leap', 'bench_press', 'broad_jump', 'three_cone', 'shuttle_run']]
y = raw1['IF_DRAFTED']

In [8]:
#Handle missing values with mean or median
X = X.fillna(X.mean())

In [9]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [10]:
y_test.unique()

array([0, 1], dtype=int64)

In [11]:
num_classes = 2
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# Original label of `2` is one-hot encoded as `01`
y_train[0]

array([0., 1.], dtype=float32)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 150
# hidden_nodes_layer3 = 50
# hidden_nodes_layer4 = 50

nn_model = tf.keras.models.Sequential()

#-------------------------------------------------------------------------------------
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# # #Third hidden layer
# nn_model.add(tf.keras.layers.Dense(
#     units=hidden_nodes_layer3, activation="relu"))

# #Fourth hidden layer
# nn_model.add(tf.keras.layers.Dense(
#     units=hidden_nodes_layer4, activation="relu"))
#-------------------------------------------------------------------------------------
# Output layer
nn_model.add(tf.keras.layers.Dense(num_classes, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               1500      
                                                                 
 dense_1 (Dense)             (None, 150)               22650     
                                                                 
 dense_2 (Dense)             (None, 2)                 302       
                                                                 
Total params: 24452 (95.52 KB)
Trainable params: 24452 (95.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=80)

Epoch 1/80


192/192 [==============================] - 11s 17ms/step - loss: 0.4861 - accuracy: 0.7458
Epoch 2/80
192/192 [==============================] - 3s 17ms/step - loss: 0.4267 - accuracy: 0.7794
Epoch 3/80
192/192 [==============================] - 3s 16ms/step - loss: 0.4127 - accuracy: 0.7858
Epoch 4/80
192/192 [==============================] - 3s 16ms/step - loss: 0.4049 - accuracy: 0.7884
Epoch 5/80
192/192 [==============================] - 3s 17ms/step - loss: 0.3993 - accuracy: 0.7918
Epoch 6/80
192/192 [==============================] - 3s 16ms/step - loss: 0.3942 - accuracy: 0.7953
Epoch 7/80
192/192 [==============================] - 3s 17ms/step - loss: 0.3892 - accuracy: 0.7982
Epoch 8/80
192/192 [==============================] - 3s 16ms/step - loss: 0.3849 - accuracy: 0.8016
Epoch 9/80
192/192 [==============================] - 3s 16ms/step - loss: 0.3820 - accuracy: 0.8013
Epoch 10/80
192/192 [==============================] - 3s 16ms/step - loss: 0.3805 - accuracy: 0.8046

In [16]:
# Evaluate the model using the training data 
# Make sure you evaluate the scaled data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

48/48 - 2s - loss: 0.7265 - accuracy: 0.7428 - 2s/epoch - 42ms/step
Loss: 0.7265289425849915, Accuracy: 0.7428198456764221


In [17]:
#Save model
# nn_model.save('Draft_model3.h5')